<h1 style="font-size:3rem;color:orange;"> OpenSeaMap </h1>

# Provide OpenSeaMap as WFS to include it in Geonode

## Data extration

In this section, we will show two different ways to download the data:

1. One way is using wget tool, a tool for downloading data through the shell. We follow the [link](https://www.jcchouinard.com/wget/#Download_Wget_on_Windows) to download wget and use it afterwards with the command:

In [ ]:
wget -O muenchen.osm "https://api.openstreetmap.org/api/0.6/map?bbox=11.54,48.14,11.543,48.145"

Navigating in shell to the file of wget where wget.exe is we run the above command where we specify the output file and we give the link to the data we want to download. 



2. Another way is to download the data from the server of OpenStreetMap in the format of .osm.pbf where the files as compresed in size. We can download for free from Geofabrik through the following link:  https://download.geofabrik.de/
and for the data of the whole planet we follow the link: https://planet.openstreetmap.org/

## Osmosis

In order to filter the downloaded data from OpenStreetMap to OpenSeaMap we will use OSMOSIS Tool. Osmosis is a command line Java application for processing OSM data ([Osmosis](https://wiki.openstreetmap.org/wiki/Osmosis#Downloading)). We can download the osmosis file from the github .zip file: [Osmosis.zip](https://github.com/openstreetmap/osmosis/releases/tag/0.48.3). 

In [ ]:
C:\Users\Mirjeta\Desktop\osmosis\bin\osmosis.bat --read-pbf C:\Users\Mirjeta\Downloads\netherlands-latest.osm.pbf --tag-filter accept-nodes seamark:type=* --write-xml C:\Users\Mirjeta\Desktop\seamarks_ned.osm 

The previous command that we execute in Shell, uses the osmosis to filter the data into Seamarks which is our area of interest. The first path is the path to the .bat file of osmosis and the second path is for the data we downloaded as .pbf format. Then in the last path, we specify the output location of processed data. 

## OSM2PGSQL

To upload our already filtered data by osmosis in the database of PostgreSQL we are going to use oms2pgsql tool.

In [ ]:
"C:\Users\Mirjeta\Desktop\osm2pgsql-bin\osm2pgsql.exe" -c -d geoserver -U geonode-postgres-admin -W -H localhost -S "C:\Users\Mirjeta\Desktop\default.style" "C:\Users\Mirjeta\Desktop\seamarks.osm"

The first path is the path to the osm2pgsql.exe of the osm2pgsql file we can download [here](https://osm2pgsql.org/doc/install.html). Then we specify the name of database and the username of the connection where we want our data to transfer. We also include the style file through the path, which we can download from [here](https://learnosm.org/en/osm-data/osm2pgsql/#get-the-style-file). Then in the end we include the path of data we provided from osmosis.
After running the command above, we have to provide the password of the user geonode-postgres-admin and the procedure of uploading the data into postgresql will be done. 

## Publishing the data into Geoserver

There are two ways we can publish the data into geoserver, manually and automatically:

1. Manually. For this, we have to connect out database of postgresql with geoserver. We add a new store where we fill out the data of database and then we publish it. 

2. Automatically. For this, we use RestAPI. The following work is based on the tutorials on the website: https://geoserver-rest.readthedocs.io/en/latest/ . From the website we follow the instructions for the installation of the library in different os, and for windows we follow the lines (we need as well gdal dependency):

In [ ]:
pip install pipwin
pipwin refresh
pipwin install gdal
pip install geoserver-rest

The geoserver-rest package is useful for the management of geospatial data in GeoServer. To use this, firstly we initialize the library and we specify the parameters of geoserver as below:

In [17]:
from geo.Geoserver import Geoserver
geo = Geoserver('http://localhost/geoserver', username='admin', password='geoserver')

To create a workspace we use the following function. It creates a workspace with the name demo.

In [19]:
geo.create_workspace(workspace='demo')

'Error: The workspace can not be created'

Now we need to create and publish featurestores and featurestore layers. In the function below for creating featurestore we give parameteres like the name of the store we want to create, part of which workspace and the details of the postgres connection. Then we publish with the next function by specifying the workspace and the store along the table of db we want to publish.

In [ ]:
geo.create_featurestore(store_name='neth_data', workspace='demo', db='geoserver', host='db', pg_user='geonode-postgres-admin', pg_password='geonode-postgres-admin-passwort')
geo.publish_featurestore(workspace='demo', store_name='neth_data', pg_table='planet_osm_point')